In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

from langchain_core.messages import HumanMessage, AIMessage
# HumanMessage: Represents the user.
# AIMessage: Represents the model's response.

from langchain_community.chat_message_histories import ChatMessageHistory
# ChatMessageHistory: A concrete, in-memory store for chat messages (user, AI, system).

from langchain_core.chat_history import BaseChatMessageHistory
# BaseChatMessageHistory: An abstract interface that defines how chat history should be stored and accessed.

from langchain_core.runnables.history import RunnableWithMessageHistory
# RunnableWithMessageHistory: A wrapper that adds automatic conversation memory to an LLM, chain, or agent.


In [2]:
load_dotenv()

GROQ_API_KEY=os.getenv("GROQ_API_KEY")
model=ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=GROQ_API_KEY)

In [4]:
model.invoke(input=[HumanMessage(content="Hello, My name is Alena")])



AIMessage(content="Hello Alena, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 42, 'total_tokens': 69, 'completion_time': 0.044296797, 'completion_tokens_details': None, 'prompt_time': 0.003729133, 'prompt_tokens_details': None, 'queue_time': 0.058379567, 'total_time': 0.04802593}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb2f3-ce2d-78b1-a0f3-ab4bedb3e1a5-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 42, 'output_tokens': 27, 'total_tokens': 69})

In [5]:
model.invoke(input=[HumanMessage(content="What is my name?")])


AIMessage(content="I don't know your name. I'm a large language model, I don't have the ability to know your personal information or recall previous conversations. Each time you interact with me, it's a new conversation. If you'd like to share your name, I'd be happy to chat with you and address you by name.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 40, 'total_tokens': 107, 'completion_time': 0.145180518, 'completion_tokens_details': None, 'prompt_time': 0.001735718, 'prompt_tokens_details': None, 'queue_time': 0.064451182, 'total_time': 0.146916236}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb2f4-4213-7b33-bf0d-053c2f3fab59-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 40, 'output_tokens': 67, 'total_tokens': 107})

In [6]:
messages=[
    
  HumanMessage(content="Hi, [my name is Chris."),
    AIMessage(content="Hello Chris! How can I help?"),
    HumanMessage(content="What is my name?")]

response=model.invoke(messages)
response.content

'Your name is Chris.'

In [8]:
# Till now our model/chatbot is not able to mainatain a context or memory 

# A dictionary to keep track of sessions in memory
store={}

# This retrieves memory from story based on session Id
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
    
#Lets add a wrapper which helps our model to use this memory
with_message_history=RunnableWithMessageHistory(
    model,get_session_history
)


In [ ]:
# this is a config object that stores configuration of a session

config1={"configurable":{"session_id":"chat1"}}

response1=with_message_history.invoke(
    [HumanMessage(content="Hi, I'm Bob and I'm a web designer")],
    config=config1
)

response1_0=with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config1
)
response1_0

AIMessage(content="Your name is Bob, and you're a web designer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 109, 'total_tokens': 122, 'completion_time': 0.030384621, 'completion_tokens_details': None, 'prompt_time': 0.004915595, 'prompt_tokens_details': None, 'queue_time': 0.058944949, 'total_time': 0.035300216}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb2fd-b381-7ce2-8422-c41c6ed0183d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 109, 'output_tokens': 13, 'total_tokens': 122})

In [ ]:
config2={"configurable":{"session_id":"chat2"}}

response2=with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config2
)
# because we changed the configuration the llm can't remember name
response2

AIMessage(content='I still don\'t know your name. You haven\'t told me what it is. If you\'d like to share your name with me, I\'d be happy to learn it and use it in our conversation. Otherwise, I\'ll just refer to you as "you" or "user".', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 115, 'total_tokens': 174, 'completion_time': 0.161490829, 'completion_tokens_details': None, 'prompt_time': 0.005108356, 'prompt_tokens_details': None, 'queue_time': 0.059679127, 'total_time': 0.166599185}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb2fe-70f1-7740-912a-65f0644b411f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 115, 'output_tokens': 59, 'total_tokens': 174})

In [14]:
config3={"configurable":{"session_id":"chat3"}}

with_message_history.invoke(
     [HumanMessage(content="Hi, I'm Harry and I'm a Business man")],
    config=config3
)
response3=with_message_history.invoke(
     [HumanMessage(content="what is my name")],
    config=config3
)
# now since chat 3 is different configuration our llm tells us about harry only
print(response3)

response3=with_message_history.invoke(
     [HumanMessage(content="Who is Bob")],
    config=config3
)
print(response3)


content='Your name is Harry.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 333, 'total_tokens': 339, 'completion_time': 0.006686497, 'completion_tokens_details': None, 'prompt_time': 0.10530212, 'prompt_tokens_details': None, 'queue_time': 0.162878869, 'total_time': 0.111988617}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_45180df409', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019bb302-0165-7110-8ab1-ba7ecd8cf4d6-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 333, 'output_tokens': 6, 'total_tokens': 339}
content='I don\'t have any information about a "Bob" in our conversation so far. You\'ve only introduced yourself as Harry, a businessman. I don\'t know anything about a Bob. Would you like to tell me about Bob?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 351, 'total_